In [17]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import json
import shutil
import sys
from pathlib import Path
from tqdm import tqdm

def find_project_root(marker=".gitignore"):
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent.resolve()
    raise FileNotFoundError(
        f"Project root marker '{marker}' not found starting from {current}")
    
root = find_project_root()

In [18]:
global_detections = pd.read_csv(f"{root}/data/global_detections.csv")
# order by frame_no
global_detections = global_detections.sort_values(by='frame_no')

In [19]:
global_detections.head(10)

,frame_no,A1_x,A1_y,A1_confidence,A2_x,A2_y,A2_confidence,B1_x,B1_y,B1_confidence,B2_x,B2_y,B2_confidence
0,1048,1507,408,0.827627,442,406,0.816876,16,374,0.797673,1892,383,0.300000
1,1049,1545,410,0.805354,441,406,0.400000,71,383,0.832680,1745,404,0.848748
2,1050,1541,413,0.400000,373,410,0.634832,60,382,0.400000,1700,411,0.832230
3,1051,1541,413,0.300000,372,409,0.400000,178,402,0.845032,1657,417,0.833924
4,1052,1666,421,0.833444,311,437,0.658793,225,410,0.840000,1614,425,0.757897
5,1053,1710,426,0.668895,310,437,0.400000,248,411,0.735438,1575,431,0.803263
6,1054,1755,432,0.832334,207,430,0.832046,311,425,0.761611,1575,423,0.400000
7,1055,1755,428,0.400000,161,436,0.838825,353,434,0.835638,1575,423,0.300000
8,1056,1795,434,0.711496,113,442,0.400000,353,433,0.400000,1486,447,0.618625
9,1057,1873,448,0.814763,62,451,0.855797,425,451,0.844724,1431,459,0.825900


In [5]:
# Dataset A
dataset_A = global_detections[['frame_no', 'A1_x', 'A1_y', 'A1_confidence', 'A2_x', 'A2_y', 'A2_confidence']].copy()
dataset_A['frame_no'] = dataset_A['frame_no'].astype(int)

# Dataset B
dataset_B = global_detections[['frame_no', 'B1_x', 'B1_y', 'B1_confidence', 'B2_x', 'B2_y', 'B2_confidence']].copy()
dataset_B['frame_no'] = dataset_B['frame_no'].astype(int)

# Save if needed
dataset_A.to_csv(f"{root}/data/global_A.csv", index=False)
dataset_B.to_csv(f"{root}/data/global_B.csv", index=False)

In [20]:
# print column names of global_detections
print(global_detections.columns)

# Rename the columns to use camera numbers instead of A1, A2, B1, B2
column_mapping = {
    'A1_x': 'cam1_x',
    'A1_y': 'cam1_y',
    'A1_confidence': 'cam1_confidence',
    'A2_x': 'cam2_x',
    'A2_y': 'cam2_y',
    'A2_confidence': 'cam2_confidence',
    'B1_x': 'cam3_x',
    'B1_y': 'cam3_y',
    'B1_confidence': 'cam3_confidence',
    'B2_x': 'cam4_x',
    'B2_y': 'cam4_y',
    'B2_confidence': 'cam4_confidence'
}

global_detections = global_detections.rename(columns=column_mapping)

Index(['frame_no', 'A1_x', 'A1_y', 'A1_confidence', 'A2_x', 'A2_y',
       'A2_confidence', 'B1_x', 'B1_y', 'B1_confidence', 'B2_x', 'B2_y',
       'B2_confidence'],
      dtype='object')


In [21]:
global_detections.head(10)

,frame_no,cam1_x,cam1_y,cam1_confidence,cam2_x,cam2_y,cam2_confidence,cam3_x,cam3_y,cam3_confidence,cam4_x,cam4_y,cam4_confidence
0,1048,1507,408,0.827627,442,406,0.816876,16,374,0.797673,1892,383,0.300000
1,1049,1545,410,0.805354,441,406,0.400000,71,383,0.832680,1745,404,0.848748
2,1050,1541,413,0.400000,373,410,0.634832,60,382,0.400000,1700,411,0.832230
3,1051,1541,413,0.300000,372,409,0.400000,178,402,0.845032,1657,417,0.833924
4,1052,1666,421,0.833444,311,437,0.658793,225,410,0.840000,1614,425,0.757897
5,1053,1710,426,0.668895,310,437,0.400000,248,411,0.735438,1575,431,0.803263
6,1054,1755,432,0.832334,207,430,0.832046,311,425,0.761611,1575,423,0.400000
7,1055,1755,428,0.400000,161,436,0.838825,353,434,0.835638,1575,423,0.300000
8,1056,1795,434,0.711496,113,442,0.400000,353,433,0.400000,1486,447,0.618625
9,1057,1873,448,0.814763,62,451,0.855797,425,451,0.844724,1431,459,0.825900
